In [265]:
import requests
import pandas as pd
import json

df1 = pd.read_excel("/Users/brijeshgoharia/Desktop/KMK CONSULTING INC/NPI_PROJECT/HCP_list1.xlsx")
df1 = df1[["First", "Name", "City", "State", "Zip Code"]]

In [266]:
df1.head(10)

,First,Name,City,State,Zip Code
0,John,Affeldt,Loma Linda,CA,92354
1,Quentin,Allen,Stuart,FL,34994
2,Zaina,Al-Mohatsab,Houston,TX,77030
3,Bita,Asghari,Needham Heights,MA,2494
4,Roya,Attar,Jackson,MI,39216
5,Brandon,Ayres,Bala Cynwood,PA,19004
6,Shamik,Bafna,Cleveland,OH,44141
7,Melissa,Barnett,Sacramento,CA,95817
8,Ken,Beckman,Westerville,OH,43082
9,Walter,Beebe,Dallas,TX,75231


In [281]:
#ERASE
#Test
response_dfs = []

for i, row in df1.iterrows():
    city =row["City"]
    city = city.split(" ")
    s = "+"
    city = s.join(city)
    
        
    state =row["State"]
    state = state.split(" ")
    s = "+"
    state = s.join(state).lower()
    
    first =row["First"]
    first = first.split(" ")
    s = "+"
    first = s.join(first)
    
    last =row["Name"]
    last = last.split(" ")
    s = "+"
    last = s.join(last)
    
    #query = """https://clinicaltables.nlm.nih.gov/api/npi_idv/v3/search?terms={}&q={}*+{}*+{}&df=NPI,name.first,name.last,addr_practice.state""".format(state,first,last,state)
    query = """https://clinicaltables.nlm.nih.gov/api/npi_idv/v3/search?terms={}&q={}+{}*+{}*&ef=NPI,name.first,name.last,addr_practice.state""".format(state,state,first,last)
    
    print(query)
    rez1 = requests.get(query)
    print(rez1)
    
    #rez1_df = pd.DataFrame(rez1)
    a = rez1.text  
    b=a[a.find('{'):a.find('}')+1]#.strip('{').split(',')
    c=eval(b)
    rez1_df=pd.DataFrame.from_dict(c)
    print(rez1_df)
    if rez1_df.empty:
        print("Empty")
        next
    else:
#         rez2=rez1_df['results'][0]
#         rez1_norm_df = pd.json_normalize(rez2)
#         rez3=rez1_norm_df['addresses']
#         rez2_norm_df = pd.json_normalize(rez3[0])
#         data1=rez1_norm_df[['number','basic.first_name','basic.last_name',]]
#         data1['state']=rez2_norm_df['state'].iloc[0]
        response_dfs.append(rez1_df)
        #rez3 = pd.DataFrame() 
       
    
    

https://clinicaltables.nlm.nih.gov/api/npi_idv/v3/search?&q=ca+john*+affeldt*&ef=NPI,name.first,name.last,addr_practice.state
<Response [400]>


SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [277]:
response_dfs

[          NPI name.first name.last addr_practice.state
 0  1275523375       JOHN   AFFELDT                  CA,
           NPI name.first name.last addr_practice.state
 0  1164483301    QUENTIN     ALLEN                  FL,
           NPI name.first name.last addr_practice.state
 0  1174974380       BITA   ASGHARI                  MA,
           NPI name.first name.last addr_practice.state
 0  1225695570       NEAL  AL-ATTAR                  MI,
           NPI name.first name.last addr_practice.state
 0  1710985460    BRANDON     AYRES                  PA,
           NPI name.first name.last addr_practice.state
 0  1073952651    MELISSA   BARNETT                  CA,
           NPI name.first name.last addr_practice.state
 0  1699731018    KENNETH   BECKMAN                  OH,
           NPI name.first name.last addr_practice.state
 0  1093742207     WALTER     BEEBE                  TX,
           NPI name.first name.last addr_practice.state
 0  1194913988       BILL      BELL     

In [273]:
#TEST
df2 = pd.concat(response_dfs, axis=0)
df3=df2.rename(columns={"name.first": "First", "name.last": "Name", "addr_practice.state": "State"})


df3['First'] = df3['First'].str.lower()
df3['Name'] = df3['Name'].str.lower()

df1['First'] = df1['First'].str.lower()
df1['Name'] = df1['Name'].str.lower()


df4 = pd.merge(df1, df3, on=["First", "Name","State"], indicator=True, how="left")
df4

,First,Name,City,State,Zip Code,NPI,_merge
0,john,affeldt,Loma Linda,CA,92354,1275523375,both
1,quentin,allen,Stuart,FL,34994,1164483301,both
2,zaina,al-mohatsab,Houston,TX,77030,NaN,left_only
3,bita,asghari,Needham Heights,MA,2494,1174974380,both
4,roya,attar,Jackson,MI,39216,NaN,left_only
...,...,...,...,...,...,...,...
263,helen,wu,Boston,MA,2116,1982669586,both
264,richard,yee,Houston,TX,77092,1205817764,both
265,elizabeth,yeu,Norfolk,VA,23502,NaN,left_only
266,gerald,zaidman,Hathorne,NY,10541,1043203995,both
